# Deep Learning 2025 — Homework Week 4
## Logistic Regression (sigmoid) & Softmax Regression
**Focus:** Implement classification from scratch.  
**Compare:** Stochastic Gradient Descent (SGD) vs Newton’s Method (for logistic).  
**Deliverables:** Filled notebook with code

---

### Learning goals
- Implement sigmoid and log-loss with numerically stable code.
- Derive/implement gradients and Hessians for logistic regression.
- Compare **SGD** vs **Newton** (speed, iterations, wall time, robustness).
- Implement **softmax regression** for multiclass classification.

---

## Setup & Utilities

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
np.random.seed(42)

# Small helpers for visualizing decision boundaries

def add_bias(X):
    return np.hstack([X, np.ones((X.shape[0], 1))])

def plot_boundary(w, X, y, title):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    Xg = np.c_[xx.ravel(), yy.ravel()]
    Xgb = add_bias(Xg)
    z = 1/(1+np.exp(-(Xgb @ w)))  # for visualization only
    Z = (z > 0.5).astype(int).reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.2, cmap="bwr")
    plt.scatter(X[:,0], X[:,1], c=y.ravel(), cmap="bwr", s=12, edgecolor='k')
    plt.title(title)
    plt.show()

# Toy datasets

def make_toy_binary(n=300, centers=((0,0), (2,2)), std=0.9):
    n1 = n//2; n2 = n - n1
    X1 = np.random.randn(n1,2)*std + np.array(centers[0])
    X2 = np.random.randn(n2,2)*std + np.array(centers[1])
    X = np.vstack([X1, X2])
    y = np.vstack([np.zeros((n1,1)), np.ones((n2,1))])
    return X, y


def make_toy_multiclass(n=450, centers=((0,0),(2.2,0),(1.1,2.0)), std=0.7):
    per = n//3
    Xs, ys = [], []
    for c, mu in enumerate(centers):
        Xc = np.random.randn(per,2)*std + np.array(mu)
        yc = np.full((per,1), c)
        Xs.append(Xc); ys.append(yc)
    X = np.vstack(Xs); y = np.vstack(ys).astype(int)
    return X, y

## Exercise 1 — Binary Logistic Regression: SGD vs Newton (from scratch)
We'll use a 2D dataset so you can visualize the boundary. You must fill core math pieces.


In [ ]:
# Data
X, y = make_toy_binary(n=300)
Xb = add_bias(X)
plt.scatter(X[:,0], X[:,1], c=y.ravel(), cmap="bwr", s=12)
plt.title("Toy binary dataset")
plt.show()

### 1.1 Sigmoid and Stable Log-Loss (Given)
We provide numerically stable sigmoid and negative log-likelihood loss (with L2 on weights, not on bias).


In [ ]:
# GIVEN

def sigmoid(z):
    # stable sigmoid
    out = np.empty_like(z)
    pos = z >= 0
    neg = ~pos
    out[pos] = 1.0 / (1.0 + np.exp(-z[pos]))
    ez = np.exp(z[neg])
    out[neg] = ez / (1.0 + ez)
    return out


def logloss(w, Xb, y, l2=0.0):
    z = Xb @ w
    p = sigmoid(z)
    eps = 1e-12
    p = np.clip(p, eps, 1-eps)
    n = Xb.shape[0]
    ll = -np.mean(y*np.log(p) + (1-y)*np.log(1-p)) + 0.5*l2*np.sum(w[:-1]**2)/n
    return ll

### 1.2 Gradients and Hessian (TODO)
Use matrix calculus to implement:
- Gradient:  $\nabla L(w) = \frac{1}{n} X_b^T (p - y) + \lambda \tilde{w} $
- Hessian:  $ H = \frac{1}{n} X_b^T S X_b + \lambda \tilde{I}$, where $ S=diag(p(1-p)) $

Hints:
- Do **not** regularize the bias (last row/entry).
- Keep shapes: `w` is (d,1), `Xb` is (n,d), `y` is (n,1).


In [ ]:
# TODO: implement grad() and hessian()

def grad(w, Xb, y, l2=0.0):
    # TODO
    raise NotImplementedError


def hessian(w, Xb, y, l2=0.0):
    # TODO (you may build S as a vector and use broadcasting; diag is fine for clarity)
    raise NotImplementedError

### 1.3 Optimizers
Implement two training routines:
- **Mini-batch SGD** (fill the update line using grad on each batch)
- **Newton's method** (full batch each step; we give damping and loop skeleton)


In [ ]:
# TODO: implement SGD optimizer

def fit_logistic_sgd(Xb, y, l2=1e-3, lr=0.2, epochs=50, batch_size=32, verbose=False):
    n, d = Xb.shape
    w = np.zeros((d,1))
    hist = []
    idx = np.arange(n)
    for ep in range(epochs):
        np.random.shuffle(idx)
        for start in range(0, n, batch_size):
            batch = idx[start:start+batch_size]
            Xb_b, y_b = Xb[batch], y[batch]
            # TODO: compute gradient on batch and update w
            # g = ...
            # w -= lr * g
            pass
        hist.append(logloss(w, Xb, y, l2=l2))
    return w, hist

In [ ]:
# TODO: implement Newton optimizer (given damping + skeleton)

def fit_logistic_newton(Xb, y, l2=1e-3, max_iter=10, damping=1e-4, verbose=False):
    n, d = Xb.shape
    w = np.zeros((d,1))
    hist = []
    for it in range(max_iter):
        # TODO: compute full-batch gradient and Hessian
        # g = grad(...)
        # H = hessian(...)
        # Solve (H + damping*I) * step = g
        # step = np.linalg.solve(H_damped, g)
        # w = w - step
        # L = logloss(w, Xb, y, l2=l2)
        # hist.append(L)
        pass
    return w, hist

### 1.4 Train & Compare
Plot loss curves for both methods and visualize the learned decision boundaries.


In [ ]:
# After you implement the functions above, uncomment:
# w_sgd, hist_sgd = fit_logistic_sgd(Xb, y, l2=1e-3, lr=0.2, epochs=50, batch_size=32)
# w_newton, hist_newton = fit_logistic_newton(Xb, y, l2=1e-3, max_iter=10, damping=1e-4)
#
# plt.plot(hist_sgd, label="SGD loss")
# plt.plot(hist_newton, label="Newton loss")
# plt.xlabel("Epoch / Iter")
# plt.ylabel("Loss")
# plt.legend(); plt.title("Convergence comparison")
# plt.show()
#
# plot_boundary(w_sgd, X, y, "Decision boundary — SGD")
# plot_boundary(w_newton, X, y, "Decision boundary — Newton")


### Short reflections (answer in Markdown)
1) Which converges in fewer iterations?  
2) Which is faster in wall-clock time on this dataset?  
3) How do batch size and learning rate affect SGD?  
4) What does damping do for Newton?


## Exercise 2 — Softmax Regression (Multiclass) with SGD (from scratch)
Implement softmax regression on a 3-class toy dataset. We'll guide numerically stable parts.


In [ ]:
Xm, ym = make_toy_multiclass()
Xmb = add_bias(Xm)
plt.scatter(Xm[:,0], Xm[:,1], c=ym.ravel(), cmap=ListedColormap(["#1f77b4","#ff7f0e","#2ca02c"]), s=12)
plt.title("Toy multiclass dataset")
plt.show()

### 2.1 Stable Softmax & Loss (Given)
We provide stable softmax and cross-entropy loss (with L2 on weights, not bias row). You will implement the gradient.


In [ ]:
# GIVEN

def softmax_logits(Z):
    Zmax = np.max(Z, axis=1, keepdims=True)
    expZ = np.exp(Z - Zmax)
    return expZ / np.sum(expZ, axis=1, keepdims=True)


def softmax_loss(W, Xb, y_int, l2=1e-3):
    Z = Xb @ W  # (n,C)
    Zmax = np.max(Z, axis=1, keepdims=True)
    logsumexp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=1, keepdims=True))
    n = Xb.shape[0]
    rows = np.arange(n)
    ll = -np.mean(Z[rows, y_int.ravel()].reshape(-1,1) - logsumexp)
    ll += 0.5*l2*np.sum(W[:-1,:]**2)/n
    return ll

### 2.2 Gradient & Training Loop (TODO)
Implement the softmax gradient and a mini-batch SGD trainer.

Hint: build one-hot targets, or compute class-wise residuals directly.


In [ ]:
# TODO: implement gradient and training

def softmax_grad(W, Xb, y_int, l2=1e-3):
    # TODO
    raise NotImplementedError


def fit_softmax_sgd(Xb, y_int, l2=1e-3, lr=0.2, epochs=120, batch_size=32, verbose=False):
    n, d = Xb.shape; C = int(y_int.max())+1
    W = np.zeros((d, C))
    hist = []
    idx = np.arange(n)
    for ep in range(epochs):
        np.random.shuffle(idx)
        for start in range(0, n, batch_size):
            batch = idx[start:start+batch_size]
            # TODO: compute gradient on batch and update W
            pass
        hist.append(softmax_loss(W, Xb, y_int, l2=l2))
    return W, hist

### 2.3 Visualize Decision Regions (Given)

In [ ]:
# GIVEN

def plot_multiclass_regions(W, X, y, title):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min,x_max,300), np.linspace(y_min,y_max,300))
    Xg = np.c_[xx.ravel(), yy.ravel()]
    Z = add_bias(Xg) @ W
    pred = np.argmax(Z, axis=1).reshape(xx.shape)
    plt.contourf(xx, yy, pred, alpha=0.25, cmap=ListedColormap(["#1f77b4","#ff7f0e","#2ca02c"]))
    plt.scatter(X[:,0], X[:,1], c=y.ravel(), cmap=ListedColormap(["#1f77b4","#ff7f0e","#2ca02c"]), s=12, edgecolor='k')
    plt.title(title)
    plt.show()

After implementing, run training and plot:

In [ ]:
# Example usage (uncomment after you implement):
# W_smx, hist_smx = fit_softmax_sgd(Xmb, ym, l2=1e-3, lr=0.2, epochs=120, batch_size=32)
# plt.plot(hist_smx); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Softmax SGD loss"); plt.show()
# plot_multiclass_regions(W_smx, Xm, ym, "Softmax regression — SGD decision regions")

## (Optional) Exercise 3 — Quick PyTorch Checkpoint
Short, practical replication using PyTorch to connect your from-scratch math to a framework.
Implement **either** binary logistic or multiclass softmax with the given skeleton.


In [ ]:
# If you want to try, uncomment the cells below.
# import torch
# from torch import nn, optim
# torch.manual_seed(42)

# # Binary (logistic):
# X_t = torch.tensor(X, dtype=torch.float32)
# y_t = torch.tensor(y, dtype=torch.float32)
# model = nn.Sequential(nn.Linear(2,1))
# criterion = nn.BCEWithLogitsLoss()
# opt = optim.SGD(model.parameters(), lr=0.2)
# for ep in range(80):
#     opt.zero_grad()
#     logits = model(X_t)
#     loss = criterion(logits, y_t)
#     loss.backward(); opt.step()
# print("Final loss (binary, torch):", loss.item())

# # Multiclass (softmax):
# Xm_t = torch.tensor(Xm, dtype=torch.float32)
# ym_t = torch.tensor(ym.squeeze(), dtype=torch.long)
# model_m = nn.Sequential(nn.Linear(2, 3))
# crit_m = nn.CrossEntropyLoss()
# opt_m = optim.SGD(model_m.parameters(), lr=0.2)
# for ep in range(120):
#     opt_m.zero_grad()
#     logits = model_m(Xm_t)
#     loss = crit_m(logits, ym_t)
#     loss.backward(); opt_m.step()
# print("Final loss (multiclass, torch):", loss.item())

## Submission
- Ensure all **TODOs** are implemented and cells run top-to-bottom
- Include your brief **reflection answers**
- Export and upload the executed `.ipynb` to Moodle
